## 1. Obtención de datos

Conéctense a la API de Banxico para descargar la serie histórica del tipo de cambio FIX.

Usen al menos 2 años de datos para entrenar.

Conserven los últimos 15 días como conjunto de prueba (test set).

In [13]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
from datetime import datetime

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

from scipy.stats import boxcox
from scipy.special import inv_boxcox

from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

In [14]:
import requests
import pandas as pd

# Tu token de Banxico
token = "b2b8aef3c6559490348dbc3e9bfbe878054a181310b39ac2e33793ac6186b216"

# Serie del tipo de cambio FIX (peso/dólar)
serie_id = "SF43718"

# Endpoint
url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_id}/datos"

# Headers con el token
headers = {"Bmx-Token": token}

# Llamada a la API
response = requests.get(url, headers=headers)
data = response.json()

# Extraer datos
datos = data["bmx"]["series"][0]["datos"]   

# Convertir a DataFrame
df = pd.DataFrame(datos)
df["Fecha"] = pd.to_datetime(df["fecha"], dayfirst=True)
df["TipoCambio"] = pd.to_numeric(df["dato"], errors="coerce")
df.set_index('Fecha', inplace=True) 
df.drop(columns=['fecha', 'dato'], inplace= True)
df.head()

,TipoCambio
Fecha,
1991-11-12,3.0735
1991-11-13,3.0712
1991-11-14,3.0718
1991-11-15,3.0684
1991-11-18,3.0673


In [15]:
print(df.index.dtype)       
print(df.shape)
print(df.head(3))
print(df.tail(3))

datetime64[ns]
(8520, 1)
            TipoCambio
Fecha                 
1991-11-12      3.0735
1991-11-13      3.0712
1991-11-14      3.0718
            TipoCambio
Fecha                 
2025-10-01     18.3477
2025-10-02     18.4843
2025-10-03     18.3902


In [16]:
import plotly.io as pio
pio.renderers.default = "browser"

fig = px.line(
    df,
    x=df.index,
    y="TipoCambio",
    title="Tipo de cambio FIX (Peso/Dólar) - Banxico",
    labels={"TipoCambio": "Pesos por USD"}
)

fig.show()

In [17]:
fecha_inicio = datetime(2021, 1, 1)
hoy = datetime.today()

dias = (hoy - fecha_inicio).days
print(f"Días entre 2021-01-01 y hoy: {dias}")

# Asegúrate de que el índice sea datetime
df.index = pd.to_datetime(df.index)

# Filtrar desde 2021-01-01 hasta hoy
fecha_inicio = "2021-01-01"
hoy = pd.Timestamp.today()

df = df.loc[fecha_inicio:hoy]

# Graficar con Plotly
fig = px.line(
    df,
    x=df.index,
    y="TipoCambio",
    title="Tipo de cambio FIX (Peso/Dólar) - Banxico",
    labels={"TipoCambio": "Pesos por USD"}
)

# Mostrar gráfico interactivo
fig.show()

Días entre 2021-01-01 y hoy: 1736


In [18]:
from sklearn.preprocessing import MinMaxScaler
# Normalizar datos
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[["TipoCambio"]])

# Función para crear ventanas
def create_dataset(series, window=5):
    X, y = [], []
    for i in range(len(series)-window):
        X.append(series[i:(i+window), 0])
        y.append(series[i+window, 0])
    return np.array(X), np.array(y)

window_size = 15
X, y = create_dataset(df_scaled, window_size)

# Separar en train y test (últimos 15 días)

n_test = 15
X_train, X_test = X[:-n_test], X[-n_test:]
y_train, y_test = y[:-n_test], y[-n_test:]

fechas_test = df.index[window_size:][-n_test:]

In [19]:
display(X_train)
display(X_test)
display(y_train)
display(y_test)

array([[0.64018385, 0.65805793, 0.61816955, ..., 0.59159554, 0.6207777 ,
        0.66528051],
       [0.65805793, 0.61816955, 0.65191143, ..., 0.6207777 , 0.66528051,
        0.69747209],
       [0.61816955, 0.65191143, 0.66294594, ..., 0.66528051, 0.69747209,
        0.67109871],
       ...,
       [0.44913183, 0.4416539 , 0.44466331, ..., 0.44174509, 0.42742759,
        0.42301379],
       [0.4416539 , 0.44466331, 0.41207047, ..., 0.42742759, 0.42301379,
        0.41942073],
       [0.44466331, 0.41207047, 0.41969432, ..., 0.42301379, 0.41942073,
        0.41152331]], shape=(1168, 15))

array([[0.41207047, 0.41969432, 0.42835777, 0.42961625, 0.4225031 ,
        0.42100751, 0.42137229, 0.43253447, 0.42890494, 0.44174509,
        0.42742759, 0.42301379, 0.41942073, 0.41152331, 0.39997811],
       [0.41969432, 0.42835777, 0.42961625, 0.4225031 , 0.42100751,
        0.42137229, 0.43253447, 0.42890494, 0.44174509, 0.42742759,
        0.42301379, 0.41942073, 0.41152331, 0.39997811, 0.39031152],
       [0.42835777, 0.42961625, 0.4225031 , 0.42100751, 0.42137229,
        0.43253447, 0.42890494, 0.44174509, 0.42742759, 0.42301379,
        0.41942073, 0.41152331, 0.39997811, 0.39031152, 0.36984752],
       [0.42961625, 0.4225031 , 0.42100751, 0.42137229, 0.43253447,
        0.42890494, 0.44174509, 0.42742759, 0.42301379, 0.41942073,
        0.41152331, 0.39997811, 0.39031152, 0.36984752, 0.36295324],
       [0.4225031 , 0.42100751, 0.42137229, 0.43253447, 0.42890494,
        0.44174509, 0.42742759, 0.42301379, 0.41942073, 0.41152331,
        0.39997811, 0.39031152, 0.36984752, 

array([0.69747209, 0.67109871, 0.70786824, ..., 0.41942073, 0.41152331,
       0.39997811], shape=(1168,))

array([0.39031152, 0.36984752, 0.36295324, 0.36939155, 0.37453491,
       0.37800029, 0.36249726, 0.38219523, 0.38483986, 0.37331291,
       0.36751295, 0.36450354, 0.36696578, 0.39188006, 0.3747173 ])

## 2. Preparación de datos

Normalicen los datos (ejemplo: MinMaxScaler).

Construyan ventanas deslizantes de tamaño definido por ustedes (ejemplo: 10, 15, 30 días).

Justifiquen el tamaño de la ventana: ¿por qué ese número de rezagos es razonable?
R = Dado que vamos a predecir 15 días, probamos un aprendizaje de 15 días por 15 días. 

In [20]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop

# ==============================
# PARÁMETROS (ajustables rápido)
# ==============================
WINDOW_SIZE = 10        # Número de columnas de ventana deslizante (len)
HIDDEN_LAYERS = [64, 32]  # Neuronas por capa oculta (puedes agregar/quitar capas)
ACTIVATION = "relu"     # Función de activación (relu, tanh, sigmoid, etc.)
OUTPUT_ACTIVATION = "linear"  # Salida para regresión
OPTIMIZER = Adam(learning_rate=0.001)  # Optimizador
EPOCHS = 50             # Número de épocas
BATCH_SIZE = 16         # Tamaño del batch
LOSS = "mse"            # Error cuadrático medio para regresión
METRICS = ["mae"]       # Métrica adicional (error absoluto medio)

# ==============================
# FUNCIONES PRINCIPALES
# ==============================

def preparar_datos(df, window_size):
    """
    Separa X e y del DataFrame.
    Asume que las últimas columnas después de 'fecha' y 'dolar' son la ventana deslizante.
    """
    X = df.iloc[:, -window_size:].values  # Últimas columnas = ventana
    y = df["dolar"].values                # Columna objetivo
    return X, y

def construir_modelo(input_dim, hidden_layers, activation, output_activation, optimizer, loss, metrics):
    """
    Construcción de la red neuronal feed-forward (FFNN).
    """
    model = Sequential()
    
    # Capa de entrada + primera oculta
    model.add(Dense(hidden_layers[0], activation=activation, input_dim=input_dim))
    
    # Capas ocultas adicionales
    for units in hidden_layers[1:]:
        model.add(Dense(units, activation=activation))
    
    # Capa de salida (regresión → 1 neurona)
    model.add(Dense(1, activation=output_activation))
    
    # Compilar
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

def entrenar_modelo(model, X, y, epochs, batch_size):
    """
    Entrena el modelo con los datos dados.
    """
    history = model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1)
    return history

# ==============================
# MAIN
# ==============================

def main(df):
    # Preparar datos
    X, y = preparar_datos(df, WINDOW_SIZE)
    
    # Construir modelo
    model = construir_modelo(
        input_dim=X.shape[1],
        hidden_layers=HIDDEN_LAYERS,
        activation=ACTIVATION,
        output_activation=OUTPUT_ACTIVATION,
        optimizer=OPTIMIZER,
        loss=LOSS,
        metrics=METRICS
    )
    
    # Entrenar modelo
    history = entrenar_modelo(model, X, y, EPOCHS, BATCH_SIZE)
    
    return model, history



## 3. Construcción del modelo FFNN

### ELEGIMOS EL MODELO 1 (Remi/Ivo) PORQUE SALE MENOS MAPE
Definan una red neuronal secuencial en Keras con:

Una o más capas densas ocultas con funciones de activación no lineales (ej. ReLU, tanh).

Una capa de salida con 1 neurona (valor del tipo de cambio).

Seleccionen y documenten sus hiperparámetros: número de capas, neuronas, epochs, batch size, optimizador.

Expliquen brevemente por qué tomaron esas decisiones.

| Componente                | Valor en tu código                                         | ¿Por qué así?                                                                                                                                                        | Qué optimiza / trade-off                                                                                     |
| ------------------------- | ---------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------ |
| **Arquitectura**          | `Dense(64, relu)` → `Dense(32, relu)` → `Dense(1, linear)` | Estructura MLP “clásica” de 2 capas ocultas que **captura no linealidades** sin irse a un modelo demasiado grande (riesgo de sobreajuste con series de 2–5k puntos). | Balancea **capacidad** vs **generalización**. Dos capas suelen ser suficiente para patrones de ventana fija. |
| **Neuronas**              | 64 → 32                                                    | Regla práctica: empezar con 2–4× el tamaño de entrada (ventana 15 ⇒ 30–60). 64/32 ofrece capacidad extra para no linealidades suaves.                                | +Capacidad para relaciones complejas. −Mayor riesgo de overfit si no regularizas.                            |
| **Función de activación** | `relu` en ocultas, `linear` en salida                      | `relu` es estable y barata; `linear` en salida porque el objetivo es **regresión** (valor continuo).                                                                 | `relu` acelera convergencia. `linear` evita saturar salidas.                                                 |
| **Optimizador**           | `adam` (default LR=1e-3)                                   | En series univariadas con MLP, **Adam** converge rápido y tolera escalas distintas tras normalización.                                                               | +Velocidad de entrenamiento. −Puede “estancarse” si LR no se ajusta.                                         |
| **Pérdida**               | `mse`                                                      | Penaliza más los errores grandes; estable para regresión.                                                                                                            | +Estabilidad. −Más sensible a outliers que MAE/Huber.                                                        |
| **Epochs**                | `EPOCHS=50`                                                | Con ventanas deslizantes y validación 10%, 50 épocas suelen ser suficientes con Adam para converger sin sobreentrenar.                                               | +Tiempo corto. −Si hay estacionalidades largas puede faltar capacidad; suple con *early stopping*.           |
| **Batch size**            | `BATCH_SIZE=16`                                            | Tamaño pequeño mejora ruido estocástico (regularización implícita) y cabe en RAM/GPU.                                                                                | +Generalización. −Entrenamiento algo más lento.                                                              |
| **Semilla (seed)**        | `SEED=40`                                                  | Reproducibilidad total (NumPy + TF).                                                                                                                                 | Comparabilidad de experimentos.                                                                              |
| **Ventana temporal**      | `window_size=15`                                           | Captura ~3 semanas hábiles (mercado FX). Buen punto medio si hay **inercia** de corto plazo.                                                                         | +Simplicidad. −Puede omitir ciclos >15 días (mensualidad, estacionalidad).                                   |


# MODELO 1

In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error

# =====================
# Parámetros globales
# =====================
SEED = 40
EPOCHS = 50
BATCH_SIZE = 16
window_size = 15  # mismo usado al crear X, y

# Reproducibilidad
np.random.seed(SEED)
tf.random.set_seed(SEED)

# =====================
# 1. Construcción modelo
# =====================
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(window_size,)))
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="linear"))  # salida escalar

model.compile(optimizer="adam", loss="mse")

# =====================
# 2. Entrenamiento
# =====================
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    verbose=1
)

# =====================
# 3. Predicciones
# =====================
# Predicciones en todo el set
y_pred_all = model.predict(np.vstack([X_train, X_test]))
y_pred_all_rescaled = scaler.inverse_transform(y_pred_all.reshape(-1, 1))
y_all_rescaled = scaler.inverse_transform(np.concatenate([y_train.reshape(-1,1), y_test.reshape(-1,1)], axis=0))


# Fechas correspondientes a y (después de la ventana deslizante)
fechas_all = df.index[window_size:]

df_pred_all = pd.DataFrame({
    "Real": y_all_rescaled.flatten(),
    "Predicho": y_pred_all_rescaled.flatten()
}, index=fechas_all)

# Predicciones solo test (últimos 15 días)
y_pred_test = model.predict(X_test)
y_pred_test_rescaled = scaler.inverse_transform(y_pred_test.reshape(-1, 1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

fechas_test = df.index[-len(y_test):]
df_pred = pd.DataFrame({
    "Real": y_test_rescaled.flatten(),
    "Predicho": y_pred_test_rescaled.flatten()
}, index=fechas_test)

# =====================
# 4. Evaluación
# =====================
mape = mean_absolute_percentage_error(y_test_rescaled, y_pred_test_rescaled) * 100
print(f"MAPE en test: {mape:.2f}%")

# =====================
# 5. Gráfica con Plotly
# =====================
fecha_inicio = "2015-01-01"
df_filtrado = df.loc[fecha_inicio:]
df_pred_all_filtrado = df_pred_all.loc[fecha_inicio:]
df_pred_filtrado = df_pred.loc[fecha_inicio:]

fig = go.Figure()

# Serie completa real
fig.add_trace(go.Scatter(x=df_filtrado.index, y=df_filtrado["TipoCambio"],
                         mode="lines",
                         name="Serie completa (Real)",
                         line=dict(color="lightgray")))

# Predicciones de todo el modelo (train + test)
fig.add_trace(go.Scatter(x=df_pred_all_filtrado.index, y=df_pred_all_filtrado["Predicho"],
                         mode="lines",
                         name="Predicho (train + test)",
                         line=dict(color="orange")))

# Últimos 15 días - reales
fig.add_trace(go.Scatter(x=df_pred_filtrado.index, y=df_pred_filtrado["Real"],
                         mode="lines+markers",
                         name="Real (últimos 15 días)",
                         line=dict(color="blue")))

# Últimos 15 días - predichos
fig.add_trace(go.Scatter(x=df_pred_filtrado.index, y=df_pred_filtrado["Predicho"],
                         mode="lines+markers",
                         name="Predicho (últimos 15 días)",
                         line=dict(color="red", dash="dot")))

fig.update_layout(title="Predicción del tipo de cambio USD/MXN con FFNN",
                  xaxis_title="Fecha",
                  yaxis_title="USD/MXN",
                  legend=dict(x=0.01, y=0.99, bordercolor="black", borderwidth=1))

Epoch 1/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0208 - val_loss: 0.0012
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - loss: 0.0012 - val_loss: 8.2067e-04
Epoch 3/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 585us/step - loss: 9.9101e-04 - val_loss: 6.5340e-04
Epoch 4/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step - loss: 9.0821e-04 - val_loss: 5.9160e-04
Epoch 5/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step - loss: 8.5154e-04 - val_loss: 5.5331e-04
Epoch 6/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - loss: 8.2613e-04 - val_loss: 5.0812e-04
Epoch 7/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - loss: 8.0756e-04 - val_loss: 4.5840e-04
Epoch 8/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - loss: 7.9182e-04 - val_loss: 4.2247e-04
Epoch 9/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - loss: 7.6348e-04 - val_loss: 4.1238e-04
Epoch 10/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step - loss: 7.4031e-04 - val_loss: 4.0803e-04
Epoch 11/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - loss: 7.2117e-04 -

# Modelo 2

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(window_size,)))
model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation = "linear"))  # salida escalar

model.compile(optimizer="adam", loss="mse")
history = model.fit(X_train, y_train, epochs=500, batch_size=36, validation_split=0.1, verbose=1)

Epoch 1/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1001 - val_loss: 0.0099
Epoch 2/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0063 - val_loss: 0.0018
Epoch 3/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0034 - val_loss: 0.0015
Epoch 4/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 921us/step - loss: 0.0019 - val_loss: 9.5331e-04
Epoch 5/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - loss: 0.0014 - val_loss: 6.7286e-04
Epoch 6/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - loss: 9.3669e-04 - val_loss: 4.9265e-04
Epoch 7/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 8.5376e-04 - val_loss: 5.3893e-04
Epoch 8/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 982us/step - loss: 8.3626e-04 - val_loss: 4.5842e-04
Epoch 9/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.9699e-04 - val_loss: 4.6592e-04
Epoch 10/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.9249e-04 - val_loss: 4.5493e-04
Epoch 11/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.8030e-04 - val_loss: 4.5762e-

In [23]:
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(-1,1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1,1))

# DataFrame comparativo
df_pred = pd.DataFrame({
    "Real": y_test_rescaled.flatten(),
    "Predicho": y_pred_rescaled.flatten()
}, index=fechas_test)

print(df_pred.head())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
               Real   Predicho
Fecha                         
2025-09-12  18.4757  18.450499
2025-09-15  18.3635  18.385088
2025-09-17  18.3257  18.281385
2025-09-18  18.3610  18.234518
2025-09-19  18.3892  18.293472


In [24]:
y_pred_all = model.predict(X)  # predicciones en todo el set (train + test)
y_pred_all_rescaled = scaler.inverse_transform(y_pred_all.reshape(-1,1))
y_all_rescaled = scaler.inverse_transform(y.reshape(-1,1))

fechas_all = df.index[window_size:]  # fechas correspondientes

df_pred_all = pd.DataFrame({
    "Real": y_all_rescaled.flatten(),
    "Predicho": y_pred_all_rescaled.flatten()
}, index=fechas_all)



# Filtrar desde 2015
fecha_inicio = "2015-01-01"
df_filtrado = df.loc[fecha_inicio:]
df_pred_all_filtrado = df_pred_all.loc[fecha_inicio:]
df_pred_filtrado = df_pred.loc[fecha_inicio:]

fig = go.Figure()

# Serie completa real
fig.add_trace(go.Scatter(x=df_filtrado.index, y=df_filtrado["TipoCambio"],
                         mode="lines",
                         name="Serie completa (Real)",
                         line=dict(color="lightgray")))

# Predicciones de todo el modelo (train + test)
fig.add_trace(go.Scatter(x=df_pred_all_filtrado.index, y=df_pred_all_filtrado["Predicho"],
                         mode="lines",
                         name="Predicho (train + test)",
                         line=dict(color="orange")))

# Últimos 15 días - reales
fig.add_trace(go.Scatter(x=df_pred_filtrado.index, y=df_pred_filtrado["Real"],
                         mode="lines+markers",
                         name="Real (últimos 15 días)",
                         line=dict(color="blue")))

# Últimos 15 días - predichos
fig.add_trace(go.Scatter(x=df_pred_filtrado.index, y=df_pred_filtrado["Predicho"],
                         mode="lines+markers",
                         name="Predicho (últimos 15 días)",
                         line=dict(color="red", dash="dot")))

fig.update_layout(title="Predicción de la TIIE con FNN",
                  xaxis_title="Fecha",
                  yaxis_title="TIIE 28 días (%)",
                  legend=dict(x=0.01, y=0.99, bordercolor="black", borderwidth=1))

fig.show()

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step
